In [29]:
import pandas as pd
movies = pd.read_csv('./data/movie_summaries_ada/movie.metadata.tsv', sep='\t', header=None)
cpi_data = pd.read_csv("./data/cpi_data/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_5994714.csv", sep=',', skiprows=4)
budget_data = pd.read_csv('./data/movies_budget/movies_metadata.csv', sep=',')

/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/2582371815.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budget_data = pd.read_csv('./data/movies_budget/movies_metadata.csv', sep=',')


In [75]:
budget_data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [97]:
#Get and clean budget data
budget = budget_data[["original_title", "release_date", "runtime", "budget", "revenue"]]
print(len(budget))
budget["revenue_budget"] = budget["revenue"]
budget = budget.drop(columns=["revenue"])
budget["budget"] = pd.to_numeric(budget["budget"], errors='coerce').dropna().astype(int)
budget = budget.dropna()
budget = budget[budget["budget"] != 0]
print(len(budget))
budget.head()

45466
8876


/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/2799707511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget["revenue_budget"] = budget["revenue"]


,original_title,release_date,runtime,budget,revenue_budget
0,Toy Story,1995-10-30,81.0,30000000.0,373554033.0
1,Jumanji,1995-12-15,104.0,65000000.0,262797249.0
3,Waiting to Exhale,1995-12-22,127.0,16000000.0,81452156.0
5,Heat,1995-12-15,170.0,60000000.0,187436818.0
6,Sabrina,1995-12-15,127.0,58000000.0,0.0


In [98]:
movies.head()

,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [99]:
#get and clean movies data
movies.columns = ['wikipedia_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres']
movie_rev = movies[["name", "release_date" , "runtime", "revenue"]]
movie_rev["revenue_ada"] = movie_rev["revenue"]
movie_rev = movie_rev.drop(columns=["revenue"])
print(len(movie_rev))
movie_rev = movie_rev.dropna()
print(len(movie_rev))
movie_rev.head()


81741
8243


/var/folders/zd/swfytvpn4w7b_b35x8v4xrgm0000gn/T/ipykernel_74117/1020249597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_rev["revenue_ada"] = movie_rev["revenue"]


,name,release_date,runtime,revenue_ada
0,Ghosts of Mars,2001-08-24,98.0,14010832.0
7,Alexander's Ragtime Band,1938-08-16,106.0,3600000.0
13,Henry V,1989-11-08,137.0,10161099.0
17,Mary Poppins,1964-08-27,139.0,102272727.0
21,New Rose Hotel,1999-10-01,92.0,21521.0


In [106]:
money_data = budget.merge(movie_rev, left_on=["original_title", "release_date", "runtime"], right_on=["name", "release_date" , "runtime"], how='left')
#drop the useless columns
money_data = money_data.drop(columns=["name"])
print(len(money_data))
money_data.head(20)

8876


,original_title,release_date,runtime,budget,revenue_budget,revenue_ada
0,Toy Story,1995-10-30,81.0,30000000.0,373554033.0,NaN
1,Jumanji,1995-12-15,104.0,65000000.0,262797249.0,262797249.0
2,Waiting to Exhale,1995-12-22,127.0,16000000.0,81452156.0,NaN
3,Heat,1995-12-15,170.0,60000000.0,187436818.0,187436818.0
4,Sabrina,1995-12-15,127.0,58000000.0,0.0,53458319.0
5,Sudden Death,1995-12-22,106.0,35000000.0,64350171.0,NaN
6,GoldenEye,1995-11-16,130.0,58000000.0,352194034.0,NaN
7,The American President,1995-11-17,106.0,62000000.0,107879496.0,NaN
8,Nixon,1995-12-22,192.0,44000000.0,13681765.0,NaN
9,Cutthroat Island,1995-12-22,119.0,98000000.0,10017322.0,NaN


In [107]:
money_data['revenue_budget'] = money_data['revenue_budget'].replace(0.0, pd.NA)

# Step 2: Create 'revenue_final' column
money_data['revenue_final'] = money_data['revenue_budget'].fillna(money_data['revenue_ada'])
money_data = money_data.drop(columns=["revenue_budget", "revenue_ada"])

# Step 3: Drop rows where 'revenue_final' is NaN or 0
money_data = money_data.dropna(subset=['revenue_final' , "budget"])
print(len(money_data))
money_data.head()

5500


,original_title,release_date,runtime,budget,revenue_final
0,Toy Story,1995-10-30,81.0,30000000.0,373554033.0
1,Jumanji,1995-12-15,104.0,65000000.0,262797249.0
2,Waiting to Exhale,1995-12-22,127.0,16000000.0,81452156.0
3,Heat,1995-12-15,170.0,60000000.0,187436818.0
4,Sabrina,1995-12-15,127.0,58000000.0,53458319.0


In [113]:
#Get CPI data
cpi_data_movies = cpi_data[cpi_data["Country Code"] == "USA"]
cpi_data_movies = cpi_data_movies.drop(columns=['Unnamed: 67'])
cpi_data_movies

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
251,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1.457976,1.070724,1.198773,1.239669,1.278912,1.585169,...,1.464833,1.622223,0.118627,1.261583,2.13011,2.442583,1.81221,1.233584,4.697859,8.0028


In [114]:
cpi_data_movies.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [119]:
# Step 1: Prepare the CPI data
cpi_long_format = cpi_data_movies.melt(id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                              var_name='Year',
                              value_name='CPI')
cpi_long_format['Year'] = cpi_long_format['Year'].astype(int)

# Step 2: Extract the year from 'release_date'
money_data['Year'] = pd.to_datetime(money_data['release_date']).dt.year

# Step 3: Merge the DataFrames
# Adjust 'Country Name' and 'Country Code' to match the country of your movie dataset
merged_df = pd.merge(money_data, cpi_long_format, how='left', on='Year')

# Step 4: Calculate Inflation-Free Prices
# Assuming you want to adjust to the latest year in your CPI data (e.g., 2022)
latest_year = 2022
latest_cpi = cpi_long_format[cpi_long_format['Year'] == latest_year]['CPI'].iloc[0]

def adjust_for_inflation(value, year_cpi):
    return value * (latest_cpi / year_cpi) if year_cpi else value

merged_df['budget_adjusted'] = merged_df.apply(lambda row: adjust_for_inflation(row['budget'], row['CPI']), axis=1)
merged_df['revenue_final_adjusted'] = merged_df.apply(lambda row: adjust_for_inflation(row['revenue_final'], row['CPI']), axis=1)

# Now merged_df contains inflation-adjusted budget and revenue_final
#merged_df['budget_adjusted'] = merged_df['budget_adjusted'].astype(int)
#merged_df['revenue_final_adjusted'] = merged_df['revenue_final_adjusted'].astype(int)


In [120]:
merged_df.head()

,original_title,release_date,runtime,budget,revenue_final,Year,Country Name,Country Code,Indicator Name,Indicator Code,CPI,budget_adjusted,revenue_final_adjusted
0,Toy Story,1995-10-30,81.0,30000000.0,373554033.0,1995,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2.80542,8.557864e+07,1.065608e+09
1,Jumanji,1995-12-15,104.0,65000000.0,262797249.0,1995,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2.80542,1.854204e+08,7.496610e+08
2,Waiting to Exhale,1995-12-22,127.0,16000000.0,81452156.0,1995,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2.80542,4.564194e+07,2.323522e+08
3,Heat,1995-12-15,170.0,60000000.0,187436818.0,1995,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2.80542,1.711573e+08,5.346863e+08
4,Sabrina,1995-12-15,127.0,58000000.0,53458319.0,1995,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2.80542,1.654520e+08,1.524963e+08
